# Downstream Applications

Finally, let's apply the trained models. We will test on several cases and proceed by votes from the cross-validated models. Several applications are

* Visualise binding specificity prediction on voxel as pymol session isosurface. For AUCG, we should aim at a multi-class multi-label situation, but for SXPR prediction is unlikely multilabel. These two cases are handled by levelling on the bit height (bit fraction) on logo diagrams.
* Visualise binding site 'projected' onto protein surface. This is a popular comparing apple-to-orange scenario when users want to adapt our halo-based prediction to prediciton of site/non-site on protein sequence.  
* Sequence Logo diagram.


We will use 16-layer Resnet models as a baseline.

In [1]:
# ============== Click Please.Imports
import sys
import glob
import gc


import random
random.seed(0)
import pandas as pd

import torch
import seaborn as sns

import matplotlib.pyplot as plt

import glob
import sys
import shutil
import tqdm



# ================
# Torch related
# ==============
import torch 
#from torch import nn



#import pytorch_lightning as pl
#import torchmetrics

# Turn on cuda optimizer
print(torch.backends.cudnn.is_available())
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
# disable debugs NOTE use only after debugging
torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)
# Disable gradient tracking
torch.no_grad()


# =============
# NN
# =================
sys.path.append('../')
from NucleicNet.DatasetBuilding.util import *
from NucleicNet.DatasetBuilding.commandReadPdbFtp import ReadBCExternalSymmetry, MakeBcClanGraph
from NucleicNet.DatasetBuilding.commandDataFetcher import FetchIndex, FetchTask, FetchDataset
from NucleicNet.DatasetBuilding.commandBenchmark import BenchmarkWrapper
from NucleicNet import Burn, Fuel
import NucleicNet.Burn.M1
import NucleicNet.Burn.util





%config InlineBackend.figure_format = 'svg'

sns.set_context("notebook")




True


# Loading pdb and featurize

Note that user should make sure that the pdb submitted does not contain non-protein parts and that the protein submitted is intended, even though we have script to retain only the following 20 canonical protein residues:
`                    "ALA","CYS","ASP","GLU","PHE","GLY", 
                    "HIS","ILE","LYS","LEU","MET","ASN", 
                    "PRO","GLN","ARG","SER","THR","VAL", 
                    "TRP","TYR"` as a basic sanitization procedure, we do not replace non-canonical amino acids, which will be left as a hole if present.

Also, do not submit a C-alpha only structure or a structure with no sidechain or heavily stubbed sidechain (particularly common practice in cryoEM!). For simplicity, we also do not do biological assembly and users should look out for rna binding at the interface of assembled unit proteins.

* The pdb files has to be put into `../LocalServerExample/` if you are using this notebook. The output is stored in a user designated folder e.g. `../ServerExample/ServerOutput/"`

# Estimated Time of Arrival 
We do not offer CPU version anymore. Runtime with a GTX 1070 for 4f3t
* Preparation: 10 minutes
* Make SXPR: 2 minutes for three 16-layer resnet
* Make AUCG: 2 minutes for three 16-layer resnet
* Total time: 14 minutes, scale almost linearly with number of atoms.

Running all the examples takes around 3-4 hours.



In [ ]:
from NucleicNet.DatasetBuilding.commandServer import Server

# NOTE This folder contains test case pdb to be sanitised.
DIR_ServerExample = '../LocalServerExample/'
UploadTestCases = sorted(glob.glob("%s/*.pdb" %(DIR_ServerExample))) 


DIR_ServerFolder = "../LocalServerExample/ServerOutputV1p1/" 
MkdirList([DIR_ServerFolder])
# NOTE Make some subfolders
for i in range(len(UploadTestCases)):
    try:
        pdbid = UploadTestCases[i].split("/")[-1].split(".")[0]
        print("Working on %s" %(pdbid))

        ServerSubFolder = DIR_ServerFolder + "%s/"%(pdbid)
        ServerC = Server(SaveCleansed = True, SaveDf = True,
                        Select_HeavyAtoms = True,
                        Select_Resname = [
                            "ALA","CYS","ASP","GLU","PHE","GLY", 
                            "HIS","ILE","LYS","LEU","MET","ASN", 
                            "PRO","GLN","ARG","SER","THR","VAL", 
                            "TRP","TYR"
                            ],
                        DIR_ServerFolder = ServerSubFolder,
                        DsspExecutable = "../NucleicNet/util/dssp",
                        DIR_ClassIndexCopy = "../Database-PDB/DerivedData/ClassIndex.pkl")

        
        ServerC.SimpleSanitise(aux_id=0, DIR_InputPdbFile=UploadTestCases[i])
        ServerC.MakeHalo(aux_id=0)
        ServerC.MakeDssp(aux_id=0)
        ServerC.MakeDummyTypi(aux_id=0)
        ServerC.MakeFeature()
        ServerC.MakeSXPR(aux_id=0, Checkpointlist = [   # NOTE These three models (with gelu) 
                                            "../Models/SXPR-9CV_SXPR-9CV/132_133/checkpoints/epoch=2-step=51700-hp_metric=0.5486971139907837.ckpt",
                                            "../Models/SXPR-9CV_SXPR-9CV/134_135/checkpoints/epoch=2-step=51473-hp_metric=0.5440343022346497.ckpt",
                                            "../Models/SXPR-9CV_SXPR-9CV/136_137/checkpoints/epoch=2-step=52381-hp_metric=0.4679257273674011.ckpt",

                                                        
                                                        ]
                                                )
        ServerC.MakeAUCG(aux_id=0,  Checkpointlist = [  # NOTE These three models (with gelu)
                                                        "../Models/AUCG-9CV_AUCG-9CV/294_295/checkpoints/epoch=7-step=26400-hp_metric=0.485228568315506.ckpt",
                                                        "../Models/AUCG-9CV_AUCG-9CV/296_297/checkpoints/epoch=7-step=25924-hp_metric=0.48591428995132446.ckpt",
                                                        "../Models/AUCG-9CV_AUCG-9CV/298_299/checkpoints/epoch=7-step=24326-hp_metric=0.5161714553833008.ckpt",

                                                        ]
                                                )
    except:
        print('Cuda Runtime error in %s' %(pdbid) )
        del ServerC
        continue
    # NOTE Downstream applications
    ServerC.Downstream_VisualisePse(aux_id=0)


    # NOTE Sequence Logo Making. We will simply take the centroid from uploaded structure.
    try:
        ServerC.Downstream_Logo(
                        DIR_InputPdbFile = UploadTestCases[i],
                        User_ProteinContactThreshold = 6.0,
                        User_SummarisingSphereRadius = 1.5)
    except AssertionError:
        print("ABORTED. Supply template base location for %s" %(pdbid))

    del ServerC


# Summary of output files
* `uploaded.pml` is the pymol script to generate `uploaded.pse`. This pse file can be opened with pymol > 2.4 `pymol uploaded.pse` and contains visualisation of the following files. The `occupancy` column in PDB format is used to store the probability and the `b-factor` column stores the bit fraction (fractional height in sequence logos.) Visualising multi-label results on voxel is difficult as the results overlap. We provide visualisation of the multi-class multi-label result with simple heuristic filters on bit fraction (to remove ambiguous base-sites) and DBSCAN (to remove spatial outliers).
* * `Result_0.4_Filtered.pdb` is the DBSCAN filtered result for visualisation of AUCG with bit fraction > 0.4. 
* * `Result_0.6_Filtered.pdb` is the DBSCAN filtered result for visualisation of PR with bit fraction > 0.6
* * `Result_OrangeSitePerAtom.pdb` is the in case users are interested in 'projection' of halo-based multi-class result for a binary classification (site/non-site) done on protein sequence positions. A per-atom heuristic score is stored in the `occupancy` column; it is calculated by counting the number of voxels corresponding to site labels. In these cases of comparing apple to orange (two different interpretable task-orientated calculations), an important remark is that taking a mean of the score over a residue may not be correct as some protein residue sidechains are buried/surfaced and only interacts with backbone; similar to adenosine binding in kinases hinge. We would suggest to take a max per residue even though it may result in false positive and user should ramp for an optimal cutoff when comparing to their binary classification. A visualisation of the per-atom score is visualised as surface in the `uploaded.pse`.
* `uploaded_*_logo_RNACColor.png` is the Sequence logo predicted by NucleicNet per base centroid location indicated by the uploaded input PDB file. `*` is the widcard for nucleotide chain ID in the file supplied by user. 


# Comments on Visualization
* `4f3t`, `2ez6`, `3k62`. These are cases we presented in the paper with visually appealing results. Personal picks are the Argonaute 2 `4f3t` and PUF `3k62`, where all the components are precisely visualised. `2ez6` is a dsRNA complex, while the phospho-sugar backbone is nicely presented, base preference is scarce.

* `7ki3` is an Argonaute 2 released Oct 2021, not in our dataset. Unlike other argonautes structure, it contains a miR-122:Site-1 three-way junction and an extra apparent binding site at G21 with loose contact with a proline (3.1 angstrom best) which the author described `despite the numerous contacts described above, SL1 and t9/G25 are both detrimental to Site-1 binding compared to a Site-1 mutant RNA with an unpaired A4 bridge between seed and supplementary paired regions. We suspect these differences may reflect tension in the Ago2:miR-122 complex, which must adopt an extended conformation to engage SL1 `. In our prediction, as expected, sites shows up satisfactorily with accurate labels, including where phosphate makes contact with the new miR-122 duplex, but in absence of specificity at G21. Note, Jordy also confirmed in email with the author Prof. Ian J. MaeRae that "t9/G25" is a typo in the statement above and should read "t9/G21".

* `5z9wLess` is a large Ebola virus nucleoprotein-RNA complex. For simplicity, only 6 assembled unit of protein is retained (,though a full result is also obtainable offline in an hour using the file `5z9w.pdb`). Interestingly, without even updating the model on our server, the binding site is predicted correctly ([submission to our server this way](http://www.cbrc.kaust.edu.sa/NucleicNet/index.html) and we have not yet even update our online server since 2018!). Two notes here. 
* * (1) As the RNA sits near the interface of assembled protein units, we should anticipate reasonable structure-based prediction when a correctly assembled structure is provided. For example, [a simple download from the pdb](https://files.rcsb.org/download/5Z9W.pdb) is not necessarily correct. And, obviously, the users should only interpret results from chain `o` and `0` with caution at the "upper" and "lower" boundary where ["symmetry mates"](https://www.ccp4.ac.uk/MG/ccp4mg_help/symmetry.html) are still missing. 
* * (2) When adapting a multi-class prediction on 3D voxels to a binary prediction on 1D sequences, cautions have to be taken as of how to 'retrieve labels' on a sequence with a simple heuristic. While an okay heuristic as we did in 2019 is simply averaging the top label in the 30 voxels nearest to a protein residue, it does not mean the heuristic is genuine for all cases (these are quick fix with no theoretical bounds!) and certainly not for cherry picking. Another decision to be made in this apple-to-orange comparison is what distances residues stand when we consider them as interacting. 

* `7eni` is a CRISPR-associated endonuclease Cas9 with an alternate location at R61 near a phosphate binding site released Apr 2022. The pdb entry still lacks the PubmedID as of June 2022 and Cas9 is absent from our AUCG dataset for various reasons (e.g. Cryo-EM, specificity not fully understood, some cas system lacks specificity in majority of regions), but sites shows up satisfactorily with accurate labels including U/G preference at position 6 and the many phospho-sugar interactions at protein helix res.41-74. 

# Comments on Sequence Logo
Finally, as a subsidiary application, we try to infer sequence logo diagrams based on structures, without invoking any training with sequencing data. Several PDB entries are presented including `1aud`, `1cvj` ,`2adc`,`2err`, `2g4b` , `2lec` ,`2py9`,  `4bs2`. Several further remarks.
* For simplicity, we display all base positions indicated in the user-supplied PDB file. A caveat of this practice is that in some cases when a non-site is predicted (e.g. the 7ki3 G21 case presented above) but a non-specific interaction shows up in the supplied PDB file, the logo diagram display should be interpreted as if it is calculated with P(Base Identity | a Base is predicted), but this does not mean the position is necessarily sequence-specific. While this simplifies scenario where users really want to take a look at what base identity is predicted even if the NucleicNet mis-classified it as non-base, which seldom occurs, caution should be taken at interpretation. A cross check on this behavior can be done by inspecting the pymol pse; if there is no voxel labeled or when the voxel is labeled as `P` or `R`, then the position may not be interepreted as sequence specific.
* Many of these structures are only available as [solution NMR-based structures](https://pdb101.rcsb.org/learn/guide-to-understanding-pdb-data/methods-for-determining-structure), where only a subset of atom coordinates are inferred from distance proxies e.g. MDS and the rest are "guesses" from forcefield minimization; this results in multiple satisfiable conformations, often more than 20. Not all of these conformers in the PDB files are necessarily precise. Examples of NMR-based PDB structures include `1aud`, `2adc`/`2ad9`, `2leb`/`2lec`, `4bs2`.  There are also structures with multiple chains. Several 'good-standing' states(0-indexing)/chains are listed below. 

* * U2AF2: `2g4b` chain B; crystal structure.
* * PABPC: `1cvj` any chain; crystal structure.
* * PCBP2: `2py9` chain F; crystal structure.
* * RBFOX1: `2err` state 0
* * SNRPA: `1aud` state 0
* * PTBP1: `2ad9` state 1 or  `2adc` state 8 chain B; state 12/13 chain C; 
* * SRSF2: `2leb` state 7; `2lec` state 12. 
* * TARDBP: `4bs2` state 9 

# Epilogue 
We have presented several downstream applications. Some further remarks.
* Ensembles. To avoid re-doing the training with even larger memory burden and hours, we simply take the three cross-validated models and let them vote for an answer. But, of course, basic [boosting/bagging type of fine-tuned training](https://en.wikipedia.org/wiki/Boosting_(machine_learning)) can be done if higher accuracy is desired. 
* Sanitisation of input. For simplicity, in the downstream applications, we only take coordinates with the alternative locaton `A` or an empty alternative location indicator. If user want an alternative location `B`, they should manually edit their PDB file or [try to do it with pymol](https://pymolwiki.org/index.php/Property_Selectors) if possible (and think thrice! Some alternative locations can have much poorer B factor/ are simply sidechain outliers such that authors put it in the dustbin.).
* Voting at spatial neighborhood. We found this technique (smoothing labels at spatially adjacent voxel) very useful in both basic training and post-processing. In the future, a message passing type of training on features can be done.


